In [1]:
import pandas as pd
import numpy as np
from scripts.pipeline_steps import step5 as s5
from scripts.notebook_operations import setup_notebook

engine, g37, transcripts, genes, mirnas = setup_notebook()


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /home/nazif/thesis/data/pyensembl/GRCh37/ensembl75/Homo_sapiens.GRCh37.75.pep.all.fa.gz.pickle


In [2]:
mut_threshold, wt_threshold = (0.5 , 0.5)
df = pd.read_parquet('data/predictions.parquet')



df = s5.filter_rows_by_thresholds(df, mut_threshold, wt_threshold)

df["is_gain"] = df.mut_prediction > df.wt_prediction

df = df.merge(genes[["gene_id", "gene_name", "gene_biotype", "in_cgc", "is_driver"]], on="gene_id", how="left")
df = df.merge(mirnas[["mirna_accession", "mirna_name", "mirna_family",  "seed"]], on="mirna_accession", how="left")

df[['in_cgc', 'is_driver']] = df[['in_cgc', 'is_driver']].fillna(False).infer_objects(copy=False)
df[['gene_id', 'mutsig', "gene_name", "gene_biotype"]] = df[['gene_id', 'mutsig', "gene_name", "gene_biotype"]].fillna("not_found").infer_objects(copy=False)


df.head()

/tmp/ipykernel_87132/4074695647.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[['in_cgc', 'is_driver']] = df[['in_cgc', 'is_driver']].fillna(False).infer_objects(copy=False)


,id,wt_prediction,mut_prediction,pred_difference,vcf_id,mirna_accession,gene_id,mutation_context,mutsig,is_intron,is_gain,gene_name,gene_biotype,in_cgc,is_driver,mirna_name,mirna_family,seed
0,PD4005a_1_12844091_G_T_MIMAT0000265,0.411319,0.724802,0.313,PD4005a,MIMAT0000265,not_found,A[C>A]A,SBS3,False,True,not_found,not_found,False,False,hsa-miR-204-5p,miR-204-5p/211-5p,GGCATAG
1,PD4005a_1_12844091_G_T_MIMAT0003301,0.289311,0.662916,0.374,PD4005a,MIMAT0003301,not_found,A[C>A]A,SBS3,False,True,not_found,not_found,False,False,hsa-miR-33b-5p,miR-33-5p,CAATGCA
2,PD4005a_1_12844091_G_T_MIMAT0004517,0.351466,0.623310,0.272,PD4005a,MIMAT0004517,not_found,A[C>A]A,SBS3,False,True,not_found,not_found,False,False,hsa-miR-106a-3p,miR-106a-3p,TAAGAAG
3,PD4005a_1_12844091_G_T_MIMAT0005882,0.550857,0.279106,-0.272,PD4005a,MIMAT0005882,not_found,A[C>A]A,SBS3,False,False,not_found,not_found,False,False,hsa-miR-548k,miR-548av-5p/548k/8054,GCAAAAT
4,PD4005a_1_12844091_G_T_MIMAT0005933,0.650570,0.375474,-0.275,PD4005a,MIMAT0005933,not_found,A[C>A]A,SBS3,False,False,not_found,not_found,False,False,hsa-miR-1277-3p,miR-1277-3p,AAATACA


In [4]:
genes

,gene_id,gene_name,chr,start,end,gene_biotype,in_cgc,is_driver
0,ENSG00000279928,DDX11L17,1,182696,184174,unprocessed_pseudogene,False,False
1,ENSG00000228037,None,1,2581560,2584533,lncRNA,False,False
2,ENSG00000142611,PRDM16,1,3069168,3438621,protein_coding,True,False
3,ENSG00000284616,None,1,5301928,5307394,lncRNA,False,False
4,ENSG00000157911,PEX10,1,2403964,2413797,protein_coding,False,False
...,...,...,...,...,...,...,...,...
63236,ENSG00000271254,None,KI270711.1,4612,29626,protein_coding,False,False
63237,ENSG00000275987,U1,KI270713.1,30437,30580,snRNA,False,False
63238,ENSG00000268674,None,KI270713.1,35407,35916,protein_coding,False,False
63239,ENSG00000277475,None,KI270713.1,31698,32528,protein_coding,False,False
